In [1]:
# %matplotlib widget

In [2]:
import os
import glob
from natsort import natsorted
from tqdm import tqdm
import itertools

import pandas as pd
import numpy as np

from nilearn import image
import nibabel as nib

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer

from scipy.stats import pearsonr, zscore, ttest_1samp
from statsmodels.stats.multitest import fdrcorrection

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
bids_dir = '/Users/emcmaho7/Desktop/SI_fMRI'
derivatives_dir = f'{bids_dir}/derivatives'
fmriprep_dir = f'{derivatives_dir}/fmriprep'
nilearn_dir = f'{derivatives_dir}/nilearn'
reliability_dir = f'{derivatives_dir}/reliability'
mask_dir = f'{derivatives_dir}/ROI_masks'
em_dir = f'{derivatives_dir}/encoding_models'

In [4]:
rois = ['EVC', 'MT', 'EBA', 'PPA', 'TOS', 'RSC', 'LOC',  'FFA', 'OFA', 'faceSTS', 'biomotion', 'SIpSTS', 'TPJ', 'DMPFC']
roi_names = ['EVC', 'MT', 'EBA', 'PPA', 'OPA', 'RSC', 'LOC',  'FFA', 'OFA', 'faceSTS', 'biomotion', 'SIpSTS', 'TPJ', 'DMPFC']

In [41]:
variable='Pearson r'
df = pd.DataFrame()

n_splits = 2
n_conditions = 200
n_condpersplit = int(n_conditions/n_splits)
n_features = len(features)
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1)

#Save the rs and null distributions
rs = np.zeros((n_subjs, len(features), tot_voxels))
for sid_ in range(4): 
    sid = str(sid_+1).zfill(2)
    y, y_inds = get_activation(z_maps[sid_, ...], None)
    n_voxels = y.shape[-1]     

    #Iterate through the different splits
    y_pred = np.zeros((n_splits, n_features, n_condpersplit, n_voxels))
    y_true = np.zeros((n_splits, n_condpersplit, n_voxels))
    test_inds = np.zeros((n_splits, n_condpersplit), dtype='int')
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        #Split the training and test data
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #save the current indices
        test_inds[i, ...] = test_index
        y_true[i, ...] = y_test

        #new X - combine the annotated features with the nuissance regressors
        X_train = np.append(X_train, control_model[train_index], axis=1)

        #Prediction
        y_pred[i, ...] = regression(X_train, y_train, X_test, features)
    y_pred = np.moveaxis(y_pred, 0, -3).reshape((n_features, n_conditions, -1))
    y_true = y_true.reshape((n_conditions, n_voxels))
    
    for ifeature, cat in enumerate(features):
        rs[sid_, ifeature, y_inds] = corr2d(y_true, y_pred[ifeature, ...])